In [ ]:
import pandas
import requests

from zipfile import ZipFile
from datetime import datetime
from io import BytesIO

In [ ]:
insee_data_path = {
    2015: {
        'met': 'https://www.insee.fr/fr/statistiques/fichier/3564182/base-cc-dipl-formation-2015.zip',
        'com': 'https://www.insee.fr/fr/statistiques/fichier/3564182/base-cc-dipl-formation-2015-COM.zip'
    },
    2014: {
        'met': 'https://www.insee.fr/fr/statistiques/fichier/3137418/base-ic-diplomes-formation-2014.zip',
        'com': 'https://www.insee.fr/fr/statistiques/fichier/3137418/base-ic-diplomes-formation-2014-com.zip'
    },
}

In [ ]:
dataset_url = insee_data_path[2015]['met']

try:
    if file_type in ('zip',):
        r = requests.get(dataset_url)
        zip_file = ZipFile(BytesIO(r.content))
        files = zip_file.namelist()
        datafile_name = files[0]
        data_file_from_zip = zip_file.read(datafile_name)
        data_file = open(datafile_name, 'wb')
        data_file.write(data_file_from_zip)
        data_file.close()
    elif file_type in ('xls', 'xlsx',):
        r = requests.get(dataset_url, proxies=proxies)
        datafile_name = dataset_url.split('/')[-1]
        data_file = open(datafile_name, 'wb')
        data_file.write(r.content)
        data_file.close()
    dataset_df = pd.read_excel(datafile_name, skiprows=4).drop([0])
    annee = dataset_url.split('-')[-1].split('.')[0]
    dataset_df.columns = [col.replace('en {}'.format(annee), '') for col in dataset_df.columns] 
except Exception as e:
    curl_status_ok = False
    error = e

In [ ]:
def check_file_extension(path, extension_types=('xls',)):
    extension = path.split('.')[-1]
    if extension in extension_types:
        return True
    else:
        return False

In [ ]:
def download_data_to_xls(path, output_dir='data'):
    if check_file_extension(path, 'zip'):
        r = requests.get(path)
        zip_file = ZipFile(BytesIO(r.content))
        files = zip_file.namelist()
        datafile_name = files[0]
        data_file_from_zip = zip_file.read(datafile_name)
        output_path = '{}/{}'.format(output_dir, datafile_name)
        data_file = open(output_path, 'wb')
        data_file.write(data_file_from_zip)
        data_file.close()
        return output_path

In [ ]:
download_data_to_xls(dataset_url, 'data/insee-diplome-formation')

## To do

- Utils
    - write output path into config file to avoid re-dl
    - auto create output subfolders depending on path
    - put data path into config file

In [ ]:
dataset_url = insee_data_path[2015]['com']
download_data_to_xls(dataset_url, 'data/insee-diplome-formation')